In [1]:
# SECURE CONNECTION: Using environment variables and dotenv for credentials
# This cell establishes a secure connection to MongoDB Atlas using:
# 1. dotenv to load credentials from a .env file (not hardcoded)
# 2. A properly formatted MongoDB Atlas URI with connection parameters
# 3. Server API version 1 for compatibility
from dotenv import load_dotenv
import os
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

# Load environment variables from .env file
load_dotenv()

username = os.getenv("MONGO_USERNAME")
password = os.getenv("MONGO_PASSWORD")
cluster = os.getenv("MONGO_CLUSTER", "cluster0.ioejg3g.mongodb.net")

uri = f"mongodb+srv://{username}:{password}@{cluster}/?retryWrites=true&w=majority&appName=Cluster0"
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("✅ Successfully connected to MongoDB!")
    print("🔒 Credentials loaded securely from .env file")
except Exception as e:
    print(f"❌ Connection failed: {e}")

✅ Successfully connected to MongoDB!
🔒 Credentials loaded securely from .env file


In [ ]:
# Database Selection and Setup
# This cell performs three operations:
# 1. Lists all available databases in the Atlas cluster
# 2. Selects the 'sample_mflix' database for use
# 3. References the 'movies' collection for subsequent queries
client.list_database_names()
db = client.sample_mflix
movies = db.movies

Question: From the `movies` collection, return the documents with the `plot` that starts with `"war"` in acending order of released date, print only title, plot and released fields. Limit the result to 5.

In [ ]:
# QUESTION 1: Aggregation Pipeline Solution
# This cell solves Question 1 using MongoDB's aggregation framework:
# - $match: Filters documents for plots starting with "war" (case-insensitive)
# - $project: Includes only the title, plot, and released fields in the output
# - $sort: Orders by released date in ascending order
# - $limit: Returns only the first 5 documents
db = client.sample_mflix
movies = db.movies
pipeline = [
    {"$match": {"plot": {"$regex": "^war", "$options": "i"}}},
    {"$project": {"title": 1, "plot": 1, "released": 1}},
    {"$sort": {"released": 1}},
    {"$limit": 5}
]
result = movies.aggregate(pipeline)
for movie in result:
    print(movie)

{'_id': ObjectId('573a1398f29313caabce9508'), 'plot': 'Warrior/pacifist Princess Nausicaè desperately struggles to prevent two warring nations from destroying themselves and their dying planet.', 'title': 'Nausicaè of the Valley of the Wind', 'released': datetime.datetime(1984, 3, 11, 0, 0)}
{'_id': ObjectId('573a1398f29313caabce91ec'), 'plot': 'Warrior/pacifist Princess Nausicaè desperately struggles to prevent two warring nations from destroying themselves and their dying planet.', 'title': 'Nausicaè of the Valley of the Wind', 'released': datetime.datetime(1984, 3, 11, 0, 0)}
{'_id': ObjectId('573a1398f29313caabcebfc6'), 'plot': 'Warlords Kagetora and Takeda each wish to prevent the other from gaining hegemony in feudal Japan. The two samurai leaders pursue one another across the countryside, engaging in massive ...', 'title': 'Heaven and Earth', 'released': datetime.datetime(1991, 2, 8, 0, 0)}
{'_id': ObjectId('573a13b5f29313caabd44f06'), 'plot': 'Warning! This synopsis contains sp

In [ ]:
# QUESTION 1: PyMongo Direct Query Solution
# This cell solves Question 1 using PyMongo's find(), sort(), and limit() methods:
# - find(): Locates documents where plot matches the regex pattern "^war" (case-insensitive)
# - sort(): Arranges results by released date in ascending order
# - limit(): Returns only the first 5 results
# - get(): Safely retrieves fields (returns None if field doesn't exist)
for movie in movies.find({"plot": {"$regex": "^war", "$options": "i"}}) \
                   .sort("released", pymongo.ASCENDING) \
                   .limit(5):
    print({
        "title": movie.get("title"),
        "plot": movie.get("plot"),
        "released": movie.get("released")
    })

{'title': 'Nausicaè of the Valley of the Wind', 'plot': 'Warrior/pacifist Princess Nausicaè desperately struggles to prevent two warring nations from destroying themselves and their dying planet.', 'released': datetime.datetime(1984, 3, 11, 0, 0)}
{'title': 'Nausicaè of the Valley of the Wind', 'plot': 'Warrior/pacifist Princess Nausicaè desperately struggles to prevent two warring nations from destroying themselves and their dying planet.', 'released': datetime.datetime(1984, 3, 11, 0, 0)}
{'title': 'Heaven and Earth', 'plot': 'Warlords Kagetora and Takeda each wish to prevent the other from gaining hegemony in feudal Japan. The two samurai leaders pursue one another across the countryside, engaging in massive ...', 'released': datetime.datetime(1991, 2, 8, 0, 0)}
{'title': 'Under the Stars', 'plot': 'Warning! This synopsis contains spoilers Bajo las estrellas (beneath the stars) features the selfish...', 'released': datetime.datetime(2007, 6, 15, 0, 0)}
{'title': 'Aliens vs. Predator

Question: Group by `rated` and count the number of movies in each.


In [ ]:
# QUESTION 2: Group by Rating
# This cell uses MongoDB's aggregation framework to group movies by their rating:
# - $group: Groups documents by the "rated" field
#   - _id: The field to group by (rated)
#   - count: Accumulator that adds 1 for each document in the group
# The result shows each unique rating and its count in the collection
pipeline = [
    {
        "$group": {
            "_id": "$rated",
            "count": {"$sum": 1}
        }
    }
]

# Execute the aggregation pipeline
results = movies.aggregate(pipeline)

# Print the results
for result in results:
    print(f"Rating: {result['_id']}, Count: {result['count']}")

Rating: GP, Count: 44
Rating: Not Rated, Count: 1
Rating: TV-PG, Count: 76
Rating: G, Count: 477
Rating: OPEN, Count: 1
Rating: R, Count: 5537
Rating: APPROVED, Count: 709
Rating: PG-13, Count: 2321
Rating: PG, Count: 1852
Rating: TV-Y7, Count: 3
Rating: TV-G, Count: 59
Rating: AO, Count: 3
Rating: Approved, Count: 5
Rating: M, Count: 37
Rating: TV-14, Count: 89
Rating: PASSED, Count: 181
Rating: TV-MA, Count: 60
Rating: None, Count: 9894


Question: Count the number of movies with 3 comments or more.


In [ ]:
# QUESTION 3: Count Movies with 3+ Comments (Performance Optimized)
# This cell counts movies with 3 or more comments using several performance techniques:
# 1. INDEX CREATION - Create index on movie_id for faster joins
index_name = db.comments.create_index("movie_id")

# 2. OPTIMIZED AGGREGATION PIPELINE
pipeline = [
    # Lookup Stage: Join movies with their comments using a pipeline approach
    {"$lookup": {
        "from": "comments",  # Join with comments collection
        "let": {"movie_id": "$_id"},  # Define variable to use in pipeline
        "pipeline": [
            # Only match relevant comments (more efficient than default lookup)
            { "$match": { "$expr": { "$eq": ["$movie_id", "$$movie_id"] } } }
        ],
        "as": "comments"  # Store matches in "comments" array
    }},
    # Filter Stage: Only keep movies with 3+ comments
    {"$match": {
        "$expr": {
            "$gte": [{ "$size": "$comments" }, 3]  # Count comments array length
        }
    }},
    # Count Stage: Count the total number of movies that match
    {"$count": "num_movies"}
]

# Execute the aggregation pipeline
results = movies.aggregate(pipeline)

# Print the results
for result in results:
    print(f"Number of movies with 3 comments or more: {result['num_movies']}")

Number of movies with 3 comments or more: 385
